# Objective 

This study aims to apply **Balanced Risk Set Matching** as a statistical method to improve causal inference in observational studies where treatment assignment is based on evolving patient conditions rather than randomization. By implementing **risk set matching**, treated patients are paired with untreated patients who had similar symptom histories up to the time of treatment, ensuring comparability at the moment of intervention. Additionally, **integer programming** is used to balance the distributions of key covariates across matched groups, minimizing bias in treatment effect estimation. This approach is applied to analyze the impact of cystoscopy and hydrodistention on interstitial cystitis symptoms, with a **sensitivity analysis** assessing the robustness of findings to hidden biases. The study ultimately aims to enhance the validity of treatment comparisons in non-randomized medical research.

# Workflow

### Step 1: Data Collection & Preprocessing
- Load or simulate the dataset, ensuring it contains treatment times, symptom histories, and follow-up measures.
- Standardize symptom measures for comparability.
- Identify treated vs. untreated patients and structure data for time-sequenced analysis.
- Output: Cleaned dataset with time-ordered symptom histories and treatment indicators.

### Step 2: Risk Set Matching
- Identify risk sets by finding untreated patients who have a similar symptom history as a treated patient up to the time of treatment.
- Ensure that future data is not used for matching.
- Compute Mahalanobis distance to measure similarity between treated and untreated patients.
- Output: Initial pool of potential matches.

### Step 3: Optimal Matching via Integer Programming
- Implement integer programming to:
    - Minimize Mahalanobis distance between treated and control pairs.
    - Ensure balanced covariate distributions across groups.
- Use network flow optimization to efficiently find the best matches.
- Output: Finalized matched dataset with treatment-control pairs.


### Step 4: Sensitivity Analysis for Hidden Bias 
- Introduce an unobserved covariate to simulate hidden biases.
- Evaluate how much hidden bias would be needed to invalidate the results.
- Conduct proportional hazards modeling to analyze potential confounders.
- Output: Bias-adjusted estimates of treatment effects.

### Step 5: Statistical Analysis & Interpretation
- Perform hypothesis testing to compare treatment vs. control groups:
    - Wilcoxon Signed-Rank Test for pairwise comparisons.
    - Permutation tests to validate significance.
    - Multivariate analysis if multiple symptom outcomes are evaluated.
- Generate visualizations (boxplots, histograms) to inspect trends in symptom changes.
- Output: Statistical validation of treatment effects.


### Step 6: Reporting & Conclusion
- Summarize methodology, findings, and potential biases.
- Present results through:
    - Summary tables (descriptive statistics).
    - Graphs & charts (symptom trends over time).
    - Sensitivity analysis conclusions (robustness of findings).
- Output: Final research paper/report with validated conclusions.



___

## Step 1: Data Collection and Preprocessing

In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import mahalanobis
from IPython.display import display, HTML
import pulp
from lifelines import CoxPHFitter
from lifelines.exceptions import ConvergenceError


# Set seed for reproducibility
np.random.seed(42)

# Number of patients to simulate
n = 100

# Create a unique patient ID for each patient
patient_ids = np.arange(1, n + 1)

# Generate a treatment indicator:
# Let's assume approximately 30% of patients are treated.
treatment = np.random.binomial(1, 0.3, n)

# Simulate treatment_time:
# - For treated patients (treatment == 1), treatment_time is drawn uniformly between 1 and 10.
# - For untreated patients (treatment == 0), we simulate a time between 11 and 20.
treatment_time = np.where(
    treatment == 1,
    np.random.uniform(1, 10, n),
    np.random.uniform(11, 20, n)
)

# Simulate symptom measures:
symptom_1 = np.random.normal(loc=50, scale=10, size=n)
symptom_2 = np.random.normal(loc=100, scale=20, size=n)
symptom_3 = np.random.normal(loc=30, scale=5, size=n)

# Simulate a follow-up outcome:
followup_outcome = 0.5 * treatment + 0.1 * symptom_1 + np.random.normal(0, 1, n)

# Create a DataFrame to hold the simulated data
data = pd.DataFrame({
    'patient_id': patient_ids,
    'treatment': treatment,
    'treatment_time': treatment_time,
    'symptom_1': symptom_1,
    'symptom_2': symptom_2,
    'symptom_3': symptom_3,
    'followup_outcome': followup_outcome
})

# Sort the DataFrame by treatment_time to simulate a time-ordered dataset
data = data.sort_values(by='treatment_time').reset_index(drop=True)

print("Sample of the simulated data (before standardization):")
display(data.head())

# --- Standardizing Symptom Measures ---
# List the symptom columns
symptom_cols = ['symptom_1', 'symptom_2', 'symptom_3']

# Initialize the scaler and fit-transform the symptom columns
scaler = StandardScaler()
data[symptom_cols] = scaler.fit_transform(data[symptom_cols])

print("\nSample of the simulated data (after standardizing symptoms):")
display(data.head())

Sample of the simulated data (before standardization):


,patient_id,treatment,treatment_time,symptom_1,symptom_2,symptom_3,followup_outcome
0,10,1,1.692819,48.852636,104.873744,29.836526,4.255557
1,53,1,2.304054,71.330334,84.817347,26.218246,7.814900
2,12,1,2.450992,58.657552,89.861136,29.554400,7.807324
3,76,1,2.569298,44.084286,97.152410,25.680046,3.663774
4,68,1,2.678667,56.296288,124.756326,23.086001,6.506929



Sample of the simulated data (after standardizing symptoms):


,patient_id,treatment,treatment_time,symptom_1,symptom_2,symptom_3,followup_outcome
0,10,1,1.692819,-0.189033,0.201526,0.127458,4.255557
1,53,1,2.304054,2.113160,-0.829741,-0.595999,7.814900
2,12,1,2.450992,0.815198,-0.570397,0.071048,7.807324
3,76,1,2.569298,-0.677413,-0.195492,-0.703609,3.663774
4,68,1,2.678667,0.573355,1.223856,-1.222275,6.506929


## Step 2: Risk Set Matching

In [14]:
# --- Assumption: 'data' is the DataFrame created in Step 1 ---
# 'data' should include:
#   - patient_id: Unique identifier for each patient.
#   - treatment: Binary indicator (1 for treated, 0 for untreated).
#   - treatment_time: Time of treatment (or censoring for untreated).
#   - symptom_1, symptom_2, symptom_3: Standardized symptom measures.
#   - followup_outcome: The follow-up outcome measure.

# Define the list of symptom columns.
symptom_cols = ['symptom_1', 'symptom_2', 'symptom_3']

# Separate the dataset into treated and untreated patients.
treated = data[data['treatment'] == 1].copy()
untreated = data[data['treatment'] == 0].copy()

# Compute the inverse covariance matrix for the symptom measures.
cov_matrix = np.cov(data[symptom_cols].values, rowvar=False)
cov_matrix += np.eye(cov_matrix.shape[0]) * 1e-6  # Add a small constant for numerical stability
cov_inv = np.linalg.inv(cov_matrix)

# Initialize a dictionary to store the risk sets.
# Each key is a treated patient's ID, and its value is a dictionary mapping
# each potential control's ID to the computed Mahalanobis distance.
risk_sets = {}

# Loop over each treated patient to build the risk sets.
for i, treated_row in treated.iterrows():
    treated_id = treated_row['patient_id']
    treated_time = treated_row['treatment_time']
    treated_vector = treated_row[symptom_cols].values

    # Identify potential controls: untreated patients whose treatment_time is later.
    potential_controls = untreated[untreated['treatment_time'] > treated_time]
    distances = {}
    
    # Compute the Mahalanobis distance for each potential control.
    for j, control_row in potential_controls.iterrows():
        control_id = control_row['patient_id']
        control_vector = control_row[symptom_cols].values
        distance = mahalanobis(treated_vector, control_vector, cov_inv)
        distances[control_id] = distance

    risk_sets[treated_id] = distances

# --- Generate and Display a Table for One Treated Patient ---

# Select the first treated patient from the risk_sets dictionary.
sample_treated_id = list(risk_sets.keys())[0]

# Prepare a list to collect rows for the table.
table_rows = []
for control_id, distance in risk_sets[sample_treated_id].items():
    table_rows.append({
        'Treated Patient': int(sample_treated_id),  # Convert to int to avoid ".0"
        'Control Patient': int(control_id),         # Convert to int to avoid ".0"
        'Mahalanobis Distance': distance
    })

# Create a DataFrame from the list of rows.
risk_set_table = pd.DataFrame(table_rows)
risk_set_table = risk_set_table.sort_values(by='Mahalanobis Distance')

# Display the table using IPython's display.
display(HTML(f"<h3>Sample Risk Set Matching Table for Treated Patient {int(sample_treated_id)}</h3>"))
display(risk_set_table)


,Treated Patient,Control Patient,Mahalanobis Distance
40,10,1,0.315674
33,10,33,0.519304
47,10,79,0.544029
62,10,46,0.589154
17,10,37,0.615874
...,...,...,...
6,10,100,2.744464
36,10,5,2.760560
12,10,40,2.819379
52,10,31,3.303858


## Step 3: Optimal Matching via Integer Programming

In [15]:
# --- Assumptions ---
# 'risk_sets' is the dictionary produced in Step 2.
# For each treated patient (key), risk_sets[key] is a dictionary where:
#    - the key is a potential control patient ID,
#    - the value is the Mahalanobis distance between the treated patient and that control.
#
# Example structure:
# risk_sets = {
#     treated_id1: {control_id_a: distance1, control_id_b: distance2, ...},
#     treated_id2: {control_id_c: distance3, control_id_d: distance4, ...},
#     ...
# }

# --- Set Up the Integer Programming Problem using PuLP ---

# Create the optimization problem: a minimization problem.
matching_problem = pulp.LpProblem("Optimal_Matching", pulp.LpMinimize)

# Create decision variables.
# x[(i,j)] = 1 if treated patient i is matched with control patient j, 0 otherwise.
x = {}
for i, controls in risk_sets.items():
    for j in controls:
        # Create a binary decision variable for each possible treated-control pair.
        x[(i, j)] = pulp.LpVariable(f"x_{int(i)}_{int(j)}", cat="Binary")

# Define the objective: minimize the total Mahalanobis distance.
matching_problem += pulp.lpSum(
    risk_sets[i][j] * x[(i, j)] for i in risk_sets for j in risk_sets[i]
), "Total_Mahalanobis_Distance"

# Constraint 1: Each treated patient must be matched to exactly one control.
for i, controls in risk_sets.items():
    matching_problem += pulp.lpSum(x[(i, j)] for j in controls) == 1, f"Match_once_treated_{i}"

# Constraint 2 (Optional): Each control can be matched to at most one treated patient.
# Build a mapping from control patient IDs to the list of treated patients that can match with it.
control_to_treated = {}
for i, controls in risk_sets.items():
    for j in controls:
        control_to_treated.setdefault(j, []).append(i)

for j, treated_list in control_to_treated.items():
    matching_problem += pulp.lpSum(x[(i, j)] for i in treated_list) <= 1, f"Control_{j}_at_most_one"

# Solve the optimization problem.
solver = pulp.PULP_CBC_CMD(msg=True)
matching_problem.solve(solver)

print("Solver Status:", pulp.LpStatus[matching_problem.status])

# --- Retrieve the Matched Pairs and Build a Table ---

matched_pairs = []
# Loop through the risk_sets and collect pairs where x[(i,j)] equals 1.
for i, controls in risk_sets.items():
    for j in controls:
        if pulp.value(x[(i, j)]) == 1:
            matched_pairs.append({
                'Treated Patient': int(i),         # Convert to int to remove any trailing .0
                'Control Patient': int(j),         # Convert to int
                'Mahalanobis Distance': risk_sets[i][j]
            })
            # print(f"Treated patient {int(i)} matched with control patient {int(j)}")

# Convert the matched pairs list into a pandas DataFrame.
matched_df = pd.DataFrame(matched_pairs)

# Optionally, sort the table by treated patient or by distance.
matched_df = matched_df.sort_values(by=['Treated Patient', 'Mahalanobis Distance'])

# Display the final matched dataset as a table.
display(HTML("<h3>Final Matched Dataset</h3>"))
display(matched_df)


Solver Status: Optimal


,Treated Patient,Control Patient,Mahalanobis Distance
19,2,58,0.183364
9,3,20,1.578098
23,8,4,0.499636
0,10,1,0.315674
2,12,25,0.626066
28,13,91,0.775335
14,26,86,0.557407
11,34,23,0.641270
29,35,22,0.708065
10,36,32,0.805556


## Step 4: Sensitivity Analysis for Hidden Bias

In [20]:
# --- Step 4: Sensitivity Analysis for Hidden Bias ---

# Assumption: The DataFrame `data` already exists.
# For demonstration, we assume `data` has these columns:
#   - 'patient_id'
#   - 'treatment': Binary indicator (1 for treated, 0 for untreated)
#   - 'followup_outcome' (placeholder from Step 1)
#
# Since Cox proportional hazards modeling requires survival data, we simulate:
#   - 'duration': follow-up time (e.g., time-to-event)
#   - 'event': indicator of whether the event occurred (1) or is censored (0)

# For reproducibility
np.random.seed(42)

n = len(data)
# Simulate duration (using an exponential distribution)
data['duration'] = np.random.exponential(scale=10, size=n)
# Simulate event indicator (e.g., 70% events observed)
data['event'] = np.random.binomial(1, 0.7, size=n)

# --- Introduce a Hidden Covariate ---
# This is an unobserved confounder.
data['hidden_bias'] = np.random.normal(loc=0, scale=1, size=n)

# --- Baseline Cox Model (γ = 0) without hidden_bias_adjusted ---
cph_baseline = CoxPHFitter()
# For gamma==0, we do not include the hidden_bias_adjusted column.
cph_baseline.fit(data[['duration', 'event', 'treatment']],
                 duration_col='duration', event_col='event')
# print("Baseline Cox PH Model (without hidden bias adjustment):")
# cph_baseline.print_summary()
# print("\n")

# --- Sensitivity Analysis ---
# We vary gamma from 0 to 2 in steps (here 21 values) and record the treatment effect.
# For gamma == 0 we already have the baseline model.
sensitivity_results = []

for gamma in np.linspace(0, 2, 21):  # 21 values: 0, 0.1, 0.2, ..., 2.0
    if gamma == 0:
        # Use the baseline model's results.
        treatment_coef = cph_baseline.params_['treatment']
        p_value = cph_baseline.summary.loc['treatment', 'p']
    else:
        # Create an adjusted hidden bias column.
        data['hidden_bias_adjusted'] = data['hidden_bias'] * gamma
        model_df = data[['duration', 'event', 'treatment', 'hidden_bias_adjusted']]
        try:
            cph = CoxPHFitter()
            cph.fit(model_df, duration_col='duration', event_col='event')
            treatment_coef = cph.params_['treatment']
            p_value = cph.summary.loc['treatment', 'p']
        except ConvergenceError:
            treatment_coef = np.nan
            p_value = np.nan
            print(f"Convergence error at gamma = {gamma:.2f}; recording NaN for treatment effect.")
    
    sensitivity_results.append({
        'Gamma (Hidden Bias Factor)': gamma,
        'Treatment Coefficient': treatment_coef,
        'p-value': p_value
    })

# Convert the results into a DataFrame.
sensitivity_df = pd.DataFrame(sensitivity_results).round(4)

# Display the sensitivity analysis table.
display(HTML("<h3>Sensitivity Analysis: Effect of Hidden Bias on Treatment Effect</h3>"))
display(sensitivity_df)


,Gamma (Hidden Bias Factor),Treatment Coefficient,p-value
0,0.0,0.0124,0.9635
1,0.1,0.0345,0.8999
2,0.2,0.0345,0.8999
3,0.3,0.0345,0.8999
4,0.4,0.0345,0.8999
5,0.5,0.0345,0.8999
6,0.6,0.0345,0.8999
7,0.7,0.0345,0.8999
8,0.8,0.0345,0.8999
9,0.9,0.0345,0.8999
